# STATISTIQUES DESCRIPTIVES

## Importation des librairies et des bases de données

In [9]:
#Import des libraries
import pandas as pd

#Ouverture des bases de données
Author_affiliation=pd.read_csv('/Users/margo/Documents/Margot/0.Polytechnique/Data-collection-visualization/Database_author_affiliation.csv')
df_match=pd.read_csv('/Users/margo/Documents/Margot/0.Polytechnique/Data-collection-visualization/Database_match_author_affiliation_JEL_code2.csv')
JEL_code=pd.read_csv('/Users/margo/Documents/Margot/0.Polytechnique/Data-collection-visualization/JEL_df1.csv')

#Finalement, les analyses de ce notebook ne porteront que sur la datafram df_match qui contient les 2 autres bases de données chargées.

In [10]:
#Pour avoir un aperçu de la dataframe df_match
df_match.head(10)
df_match.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 748225 entries, 0 to 748224
Data columns (total 16 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   JEL Subject           748225 non-null  object 
 1   Title                 748225 non-null  object 
 2   Journal               284224 non-null  object 
 3   Year                  747317 non-null  float64
 4   Type                  748225 non-null  object 
 5   URL                   748225 non-null  object 
 6   Author(s)             748225 non-null  object 
 7   author_name           748225 non-null  object 
 8   co_authors            444119 non-null  object 
 9   short_id              748225 non-null  object 
 10  institution           748225 non-null  object 
 11  share_pct             748225 non-null  int64  
 12  location              748225 non-null  object 
 13  repec_institution_id  748225 non-null  object 
 14  author_url            748225 non-null  object 
 15  

## Classement des JEL code par nombre d'articles publiés 

In [11]:
#Recodage des codes JEL en noms complets
jel_mapping = {
    'A': 'General Economics and Teaching',
    'B': 'History of Economic Thought, Methodology, and Heterodox Approaches',
    'C': 'Mathematical and Quantitative Methods',
    'D': 'Microeconomics',
    'E': 'Macroeconomics and Monetary Economics',
    'F': 'International Economics',
    'G': 'Financial Economics',
    'H': 'Public Economics',
    'I': 'Health, Education, and Welfare',
    'J': 'Labor and Demographic Economics',
    'K': 'Law and Economics',
    'L': 'Industrial Organization',
    'M': 'Business Administration and Business Economics; Marketing; Accounting; Personnel Economics',
    'N': 'Economic History',
    'O': 'Economic Development, Innovation, Technological Change, and Growth',
    'P': 'Economic Systems',
    'Q': 'Agricultural and Natural Resource Economics; Environmental and Ecological Economics',
    'R': 'Urban, Rural, Regional, Real Estate, and Transportation Economics',
    'Y': 'Miscellaneous Categories',
    'Z': 'Other Special Topics'
}

#Calcul initial
stats_jel = (
    df_match.groupby("JEL Subject")["URL"]
    .nunique()
    .reset_index()
    .rename(columns={"URL": "Count", "JEL Subject": "Code"})
)
stats_jel["Field"] = stats_jel["Code"].map(jel_mapping)

#Remplacement des & par \& pour LaTeX
stats_jel["Field"] = stats_jel["Field"].str.replace("&", "\\&", regex=False)

total_count = stats_jel["Count"].sum()
stats_jel["Share (%)"] = (stats_jel["Count"] / total_count * 100)

#Séparation entre >= 5% et < 5%
top_fields = stats_jel[stats_jel["Share (%)"] >= 5].copy()
others_fields = stats_jel[stats_jel["Share (%)"] < 5].copy()

#Création de la ligne "Others"
others_row = pd.DataFrame({
    'Code': ['-'],
    'Field': ['Other categories (< 5%)'],
    'Count': [others_fields['Count'].sum()],
    'Share (%)': [others_fields['Share (%)'].sum()]
})

#Assemblage final et arrondi à 1 décimale
df_final = pd.concat([top_fields.sort_values("Count", ascending=False), others_row], ignore_index=True)
df_final["Share (%)"] = df_final["Share (%)"].round(1)

#Ajout de la ligne TOTAL
total_row = pd.DataFrame({
    'Code': [''],
    'Field': ['\\textbf{Total}'],
    'Count': [df_final['Count'].sum()],
    'Share (%)': [df_final['Share (%)'].sum().round(0)] 
})
df_final = pd.concat([df_final, total_row], ignore_index=True)

#Affichage du code LaTeX (escape=False est important pour le \textbf et \&)
print(df_final.to_latex(index=False, escape=False))

\begin{tabular}{lrlr}
\toprule
Code & Count & Field & Share (%) \\
\midrule
D & 44998 & Microeconomics & 13.500000 \\
C & 44948 & Mathematical and Quantitative Methods & 13.400000 \\
J & 34887 & Labor and Demographic Economics & 10.400000 \\
G & 31085 & Financial Economics & 9.300000 \\
I & 26545 & Health, Education, and Welfare & 7.900000 \\
F & 26540 & International Economics & 7.900000 \\
H & 23086 & Public Economics & 6.900000 \\
L & 19970 & Industrial Organization & 6.000000 \\
Q & 18642 & Agricultural and Natural Resource Economics; Environmental and Ecological Economics & 5.600000 \\
E & 18041 & Macroeconomics and Monetary Economics & 5.400000 \\
- & 45501 & Other categories (< 5%) & 13.600000 \\
 & 334243 & \textbf{Total} & 100.000000 \\
\bottomrule
\end{tabular}



## Parmi les tops institutions et les top JEL code, quelle est la répartition de leurs sujets de recherche

In [12]:
# On s'assure que share_pct est numérique (on enlève les éventuels symboles %)
if df_match['share_pct'].dtype == 'object':
    df_match['share_pct'] = df_match['share_pct'].str.replace('%', '', regex=False)

#Conversion en numérique, les erreurs deviennent des NaN
df_match['share_pct'] = pd.to_numeric(df_match['share_pct'], errors='coerce')

#Remplissage des vides par 100 (part complète par défaut)
df_match['share_pct'] = df_match['share_pct'].fillna(100)

#Calcul du poids (100% = 1 article équivalent)
df_match['weight'] = df_match['share_pct'] / 100

#Somme des poids par institution et par Code JEL
inst_jel_weight = df_match.groupby(['institution', 'JEL Subject'])['weight'].sum().reset_index()

#Identifier le Top 10 des institutions (en production totale pondérée)
top_10_inst = inst_jel_weight.groupby('institution')['weight'].sum().sort_values(ascending=False).head(10).index

#Pivot et normalisation
pivot_table = inst_jel_weight[inst_jel_weight['institution'].isin(top_10_inst)].pivot(
    index='institution', columns='JEL Subject', values='weight'
).fillna(0)

#Diviser chaque ligne par sa somme pour avoir des pourcentages (Profil de spécialisation)
specialization_pct = pivot_table.div(pivot_table.sum(axis=1), axis=0) * 100

#Sélection des colonnes JEL principales et formatage LaTeX
main_columns = ['D', 'C', 'J', 'G', 'I', 'F', 'E', 'H']

#On vérifie que ces colonnes existent dans le pivot pour éviter une erreur
main_columns = [c for c in main_columns if c in specialization_pct.columns]

final_table = specialization_pct[main_columns].round(1)

# accourcir les noms d'institutions pour que ça tienne sur la slide
final_table.index = [i[:30] + "..." if len(str(i)) > 30 else i for i in final_table.index]

#Export LaTeX propre
print(final_table)
print(final_table.to_latex(escape=False))

JEL Subject                           D     C     J     G     I     F     E  \
Banca d'Italia                     12.2  13.7   8.4  20.2   3.8  11.1   4.5   
Banco de España                    10.8  11.7  11.6  17.9   5.0   9.1  12.5   
DIW Berlin (Deutsches Institut...  13.7   8.6  12.8   4.4   6.9  11.1  11.6   
European Central Bank              10.5  14.1   2.9  27.8   0.6  14.9  16.8   
Institute of Labor Economics (...  13.8   9.2  29.2   2.6  15.6   5.3   2.6   
Joint Research Centre European...  13.2  17.7   7.4   6.6   7.1   8.0   3.9   
Leibniz-Zentrum für Europäisch...  18.5  10.6  11.4   4.5   8.6   4.5   2.5   
Paris School of Economics          20.8   9.2  13.7   3.8   9.3  10.0   4.2   
Wiener Institut für Internatio...   3.2   4.0  14.1   3.9   6.5  22.9  13.3   
ifo Institut - Leibniz-Institu...  15.4   7.4  12.8   3.1   8.8   9.5   4.3   

JEL Subject                           H  
Banca d'Italia                      7.2  
Banco de España                     7.3  
DIW 

## Top institutions UK (weighted)

In [13]:
#Filtre pour ne garder uniquement le Royaume-Uni
df_uk = df_match[df_match['location'].str.contains('United Kingdom|UK', case=False, na=False)].copy()

# Calcul des stats par institution britannique
uk_inst_stats = df_uk.groupby('institution').agg({
    'short_id': 'nunique', #Nombre d'auteurs uniques
    'weight': 'sum'        #Volume d'articles pondéré
}).reset_index()

uk_inst_stats.columns = ['Institution', 'Authors', 'Weighted Articles']

#Calcul des parts (%) au sein du UK
total_uk_authors = df_uk['short_id'].nunique()
total_uk_weighted_articles = df_uk['weight'].sum()

uk_inst_stats['Author Share (%)'] = (uk_inst_stats['Authors'] / total_uk_authors * 100).round(1)
uk_inst_stats['Article Share (%)'] = (uk_inst_stats['Weighted Articles'] / total_uk_weighted_articles * 100).round(1)

#Top 10 des institutions britanniques par volume de production
top_10_uk = uk_inst_stats.sort_values(by='Weighted Articles', ascending=False).head(10)

#Nettoyage des noms pour le tableau
top_10_uk['Institution'] = top_10_uk['Institution'].apply(lambda x: x[:40] + "..." if len(x) > 40 else x)

print(f"Total Weighted Articles in UK: {total_uk_weighted_articles:.1f}")
print(top_10_uk)

Total Weighted Articles in UK: 73758.0
                                     Institution  Authors  Weighted Articles  \
162  Department of Economics University of Wa...       79            2848.57   
8                                Bank of England      172            2746.21   
150  Department of Economics Oxford Universit...      110            2458.25   
78   Centre for Economic Policy Research (CEP...      349            2011.33   
76   Centre for Economic Performance (CEP) Lo...       80            1812.16   
200  Economics Department London School of Ec...       45            1716.04   
156  Department of Economics University Colle...       52            1569.44   
231  Faculty of Economics University of Cambr...       56            1560.26   
352  School of Economics and Finance Queen Ma...       58            1521.87   
159  Department of Economics University of Bi...       46            1201.44   

     Author Share (%)  Article Share (%)  
162               2.2                

## Top institutions EU et UK (weighted)

In [14]:
df_global = df_match.copy()

#Calcul des stats par institution
global_inst_stats = df_global.groupby('institution').agg({
    'short_id': 'nunique', #Nombre d'auteurs uniques
    'weight': 'sum'        #Volume d'articles pondéré
}).reset_index()

global_inst_stats.columns = ['Institution', 'Authors', 'Weighted Articles']

#Calcul des parts (%) par rapport au total
total_global_authors = df_global['short_id'].nunique()
total_global_weighted_articles = df_global['weight'].sum()

global_inst_stats['Author Share (%)'] = (global_inst_stats['Authors'] / total_global_authors * 100).round(1)
global_inst_stats['Article Share (%)'] = (global_inst_stats['Weighted Articles'] / total_global_weighted_articles * 100).round(1)

#Top 15 institutions par volume de production (Weighted articles)
#On passe à 15 pour voir si des institutions UK intègrent le classement dominé par françaises/allemandes
top_global = global_inst_stats.sort_values(by='Weighted Articles', ascending=False).head(15)

#Nettoyage des noms pour le tableau
top_global['Institution'] = top_global['Institution'].apply(lambda x: x[:45] + "..." if len(x) > 45 else x)

print(f"Total Weighted Articles (Global): {total_global_weighted_articles:.1f}")
print(top_global)

Total Weighted Articles (Global): 456720.2
                                           Institution  Authors  \
524   DIW Berlin (Deutsches Institut für Wirtschaft...      113   
2747  ifo Institut - Leibniz-Institut für Wirtschaf...      172   
86                                      Banca d'Italia      359   
1885                Institute of Labor Economics (IZA)      896   
2022         Joint Research Centre European Commission      284   
87                                     Banco de España      119   
1185                             European Central Bank      228   
2335                         Paris School of Economics      172   
2665  Wiener Institut für Internationale Wirtschaft...       42   
2188  Leibniz-Zentrum für Europäische Wirtschaftsfo...      168   
966   Dipartimento di Scienze Economiche Alma Mater...      110   
870                                Deutsche Bundesbank      151   
809      Department of Economics University of Warwick       79   
91                 

## Pour le top 10 des institutions, quel est le top 3 de leurs sujets de recherche

In [15]:
# On transforme le format "large" en format "long" pour trier plus facilement
top_fields = specialization_pct.stack().reset_index()
top_fields.columns = ['institution', 'Code', 'Share']

#On garde les 3 meilleurs par institution
top_3_per_inst = top_fields.sort_values(['institution', 'Share'], ascending=[True, False]).groupby('institution').head(3)

#Ajout des descriptions JEL (en utilisant votre dictionnaire jel_mapping)
top_3_per_inst['Field'] = top_3_per_inst['Code'].map(jel_mapping)

#On concatène le nom et le % pour l'affichage : "Microeconomics (20.8%)"
top_3_per_inst['Display'] = top_3_per_inst['Field'] + " (" + top_3_per_inst['Share'].round(1).astype(str) + "%)"

#On pivote pour avoir Rank 1, 2, 3 en colonnes
final_top3 = top_3_per_inst.groupby('institution')['Display'].apply(list).apply(pd.Series)
final_top3.columns = ['Rank 1', 'Rank 2', 'Rank 3']

print(final_top3)
print(final_top3.to_latex(escape=False))

                                                                                           Rank 1  \
institution                                                                                         
Banca d'Italia                                                        Financial Economics (20.2%)   
Banco de España                                                       Financial Economics (17.9%)   
DIW Berlin (Deutsches Institut für Wirtschaftsf...                         Microeconomics (13.7%)   
European Central Bank                                                 Financial Economics (27.8%)   
Institute of Labor Economics (IZA)                        Labor and Demographic Economics (29.2%)   
Joint Research Centre European Commission           Mathematical and Quantitative Methods (17.7%)   
Leibniz-Zentrum für Europäische Wirtschaftsfors...                         Microeconomics (18.5%)   
Paris School of Economics                                                  Microeconomics (

## Focus sur les tops institutions à Paris

In [16]:
#Nettoyage et calcul du poids
df_match['share_pct'] = pd.to_numeric(df_match['share_pct'].astype(str).str.replace('%', ''), errors='coerce').fillna(100)
df_match['weight'] = df_match['share_pct'] / 100

#Filtrage Géographique (Paris)
mask_paris = (df_match['institution'].str.contains('Paris', case=False, na=False)) | \
             (df_match['location'].str.contains('Paris', case=False, na=False))
df_paris = df_match[mask_paris].copy()

#Calcul des statistiques par institution
paris_stats = df_paris.groupby('institution').agg({
    'short_id': 'nunique', # Nombre d'auteurs
    'weight': 'sum'        # Volume d'articles
}).reset_index()

paris_stats.columns = ['Institution', 'Authors', 'Weighted Articles']

#Calcul des Parts (%)
total_paris_authors = df_paris['short_id'].nunique()
total_paris_weighted_articles = df_paris['weight'].sum()

#Part des auteurs
paris_stats['Author Share (%)'] = (paris_stats['Authors'] / total_paris_authors * 100).round(1)
#Part des articles
paris_stats['Article Share (%)'] = (paris_stats['Weighted Articles'] / total_paris_weighted_articles * 100).round(1)

#Sélection du Top 10 basé sur le nombre d'auteurs
top_10_paris = paris_stats.sort_values(by='Authors', ascending=False).head(10)

#Nettoyage des noms pour LaTeX
top_10_paris['Institution'] = top_10_paris['Institution'].apply(lambda x: x[:35] + "..." if len(x) > 35 else x)

print(top_10_paris)
print(top_10_paris.to_latex(index=False))

                                Institution  Authors  Weighted Articles  \
110               Paris School of Economics      172            3506.00   
3                          Banque de France      158            2104.87   
59   EconomiX Université Paris-Nanterre ...       85            1118.36   
12   Centre d'Économie de la Sorbonne Un...       74             842.08   
60   Economics Department Organisation d...       63            1487.63   
11   Centre d'Économie de l'Université P...       47             510.75   
81   Institut de Préparation à l'Adminis...       46            1059.52   
38   Department of Economics Sciences éc...       46            1252.15   
134  Équipe de Recherche sur l'Utilisati...       44             356.74   
98   Laboratoire d'Économie de Dauphine ...       41             514.90   

     Author Share (%)  Article Share (%)  
110              12.0               14.7  
3                11.0                8.8  
59                5.9                4.7  
12

## Focus sur les tops institutions en Île de France

In [17]:
#Filtrer pour garder uniquement la localisation France
# On utilise str.contains pour capter "France" même s'il y a d'autres détails
df_france = df_match[df_match['location'].str.contains('France', case=False, na=False)].copy()

#Extraire la ville
#Souvent dans RePEc, le format est "Ville, France". On va isoler ce qui est avant la virgule.
df_france['city'] = df_france['location'].apply(lambda x: x.split(',')[0].strip())

#Créer la liste des villes uniques et compter le nombre d'institutions par ville
villes_stats = df_france.groupby('city')['institution'].nunique().sort_values(ascending=False).reset_index()
villes_stats.columns = ['Ville', 'Nombre d\'Institutions']

#Afficher la liste complète
print(f"Nombre de villes identifiées en France : {len(villes_stats)}")
print(villes_stats)

Nombre de villes identifiées en France : 72
                   Ville  Nombre d'Institutions
0                  Paris                    108
1               Toulouse                     17
2                   Lyon                     10
3                  Lille                      9
4            Montpellier                      8
..                   ...                    ...
67  Marseille / Bordeaux                      1
68            Metz/Nancy                      1
69      Nancy/Strasbourg                      1
70              Nanterre                      1
71                  Évry                      1

[72 rows x 2 columns]


In [ ]:
#Définition de la liste complète des pôles IDF identifiés
villes_idf = [
    'Paris', 'Créteil', 'Saint-Aubin', 'Cergy-Pontoise', 'Palaiseau', 
    'Fontainebleau', 'Jouy-en-Josas', 'Saint-Denis', 'Noisy le Grand', 
    'Sceaux', 'Paris-Grignon', 'Rueil-Malmaison', 'Marne', 'Noisy-le-Grand', 
    'Courbevoix', 'Cachan', 'Châtenay-Malabry', 'Courbevoie', 
    'Nogent-sur-Marne', 'Nanterre', 'Évry'
]

#Création du masque et filtrage
#On crée une expression régulière : 'Paris|Créteil|Saint-Aubin|...'
regex_idf = '|'.join(villes_idf)
df_île_france = df_match[df_match['location'].str.contains(regex_idf, case=False, na=False)].copy()

#Calcul des statistiques par institution
île_france_stats = df_île_france.groupby('institution').agg({
    'short_id': 'nunique', # Nombre d'auteurs uniques
    'weight': 'sum'        # Volume d'articles pondéré
}).reset_index()

île_france_stats.columns = ['Institution', 'Authors', 'Weighted Articles']

#Calcul des Parts (%) au sein de l'IDF
total_idf_authors = df_île_france['short_id'].nunique()
total_idf_weighted_articles = df_île_france['weight'].sum()

#Part des auteurs
île_france_stats['Author Share (%)'] = (île_france_stats['Authors'] / total_idf_authors * 100).round(1)
#Part des articles
île_france_stats['Article Share (%)'] = (île_france_stats['Weighted Articles'] / total_idf_weighted_articles * 100).round(1)

#Sélection du Top 10 basé sur le nombre d'auteurs
top_10_île_france = île_france_stats.sort_values(by='Authors', ascending=False).head(10)

#Nettoyage des noms pour l'affichage
top_10_île_france['Institution'] = top_10_île_france['Institution'].apply(lambda x: x[:40] + "..." if len(x) > 40 else x)

print(f"Nombre total d'auteurs uniques en IDF : {total_idf_authors}")
print(top_10_île_france)
print(top_10_île_france.to_latex(index=False))


Nombre total d'auteurs uniques en IDF : 1594
                                     Institution  Authors  Weighted Articles  \
120                    Paris School of Economics      172            3506.00   
3                               Banque de France      158            2104.87   
22   Centre de Recherche en Économie et Stati...       92            1917.39   
66   EconomiX Université Paris-Nanterre (Pari...       85            1118.36   
12   Centre d'Économie de la Sorbonne Univers...       74             842.08   
68   Economics Department Organisation de Coo...       63            1487.63   
11   Centre d'Économie de l'Université Paris-...       47             510.75   
41   Department of Economics Sciences économi...       46            1252.15   
132  Théorie Économique, Modélisation, Applic...       46             611.95   
90   Institut de Préparation à l'Administrati...       46            1059.52   

     Author Share (%)  Article Share (%)  
120              10.8          